**NOTE ⚠** In order to run the code, go to `Runtime` >> and select the `Run all` options.


# **Libraries and Files Installation**
1. **Installation**: We start by installing the required packages, particularly `pytorch-lightning` version 1.8.3.

2. **Imports**: We import necessary modules and libraries, including: `AutoTokenizer`, `AutoModel`, `transformers`, `numpy`, `pandas`, `LabelEncoder`, etc ...

3. **Google Drive Authentication**: It authenticates with Google Drive to access specific files using the PyDrive library. This is done through the `auth.authenticate_user()` and related code.

4. **File Retrieval**: We retreive necessary files from Google Drive using their unique IDs. These files include:
   - A model checkpoint file (`best-checkpoint-v1.ckpt`).
   - A label encoding file (`labelencoder.json`).
   - Testing samples file `samples_id`.

In [ ]:
!pip install pytorch-lightning==1.8.3 --quiet

In [ ]:
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer
from transformers import AutoModel, AdamW, get_linear_schedule_with_warmup
import pytorch_lightning as pl
import torch
import json
import torch.nn as nn
from pytorch_lightning.callbacks import ModelCheckpoint

import os
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from typing import Optional
import json

In [ ]:
from pydrive.auth import GoogleAuth
from google.colab import drive
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [ ]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

checkpoint_id = '1eeWe9TqZ-vaPXZMf7gxHYnsuDDnlVcHI'
encoding_id = '1sIoBeqALitCAbQOL24biTuXcaci7DjBy'
samples_id = '1QfMlJd_DNTJVE4bMOIw_K1kAnJB2IY6z'

checkpoint = drive.CreateFile({'id': checkpoint_id})
checkpoint.GetContentFile('best-checkpoint-v1.ckpt')

encoding = drive.CreateFile({'id': encoding_id})
encoding.GetContentFile('labelencoder.json')

samples = drive.CreateFile({'id': samples_id})
samples.GetContentFile('samples_for_testing.txt')

# **Definitions**
Here, we define classes and functions needing to generate the recommendations.

In [ ]:
class BertTextClassifier(pl.LightningModule):
    def __init__(
        self,
        bert_model: str,
        n_classes: int,
        lr: float = 2e-5,
        label_column: str = "label",
        n_training_steps=None,
        outputdir: str = "outputs",

    ):
        super().__init__()
        self.bert_model = bert_model
        self.label_column = label_column
        self.bert = AutoModel.from_pretrained(bert_model, return_dict=True)
        self.classifier = nn.Linear(self.bert.config.hidden_size, n_classes)
        self.n_classes = n_classes
        self.n_training_steps = n_training_steps
        self.criterion = nn.CrossEntropyLoss()
        self.lr = lr
        self.average_training_loss = None
        self.average_validation_loss = None
        self.outputdir = outputdir

    def forward(self, input_ids, attention_mask, labels=None):
        output = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        output = self.classifier(output.pooler_output)
        loss = 0
        if labels is not None:
            loss = self.criterion(output, labels.long())
        return loss, output

    def training_step(self, batch, batch_idx):
        input_ids = batch["input_ids"]
        attention_mask = batch["attention_mask"]
        labels = batch["labels"]
        loss, outputs = self(input_ids, attention_mask, labels)
        outputs = torch.argmax(outputs, dim=1)
        self.log("train_loss", loss, prog_bar=True, logger=True, batch_size=len(batch))
        return {"loss": loss, "predictions": outputs, "labels": labels}

    def validation_step(self, batch, batch_idx):
        input_ids = batch["input_ids"]
        attention_mask = batch["attention_mask"]
        labels = batch["labels"]
        loss, outputs = self(input_ids, attention_mask, labels)
        outputs = torch.argmax(outputs, dim=1)

        self.log("val_loss", loss, prog_bar=True, logger=True, batch_size=len(batch))

        return loss

    def test_step(self, batch, batch_idx):
        input_ids = batch["input_ids"]
        attention_mask = batch["attention_mask"]
        labels = batch["labels"]
        loss, outputs = self(input_ids, attention_mask, labels)
        outputs = torch.argmax(outputs, dim=1)
        self.log("test_loss", loss, prog_bar=True, logger=True, batch_size=len(batch))
        return loss

    def configure_optimizers(self):
      optimizer = torch.optim.Adam(self.parameters(), lr=self.lr)
      return [optimizer]

class BERTmodel:

    def __init__(self) -> None:
        print("BERTmode created")

    def from_pretrained(self, model_name="roberta-base", tokenizer=None) -> None:
        if tokenizer is not None:
            self.tokenizer = tokenizer
        else:
            self.tokenizer = AutoTokenizer.from_pretrained(f"{model_name}")
            self.model = AutoModel.from_pretrained(
                f"{model_name}", return_dict=True
            )

    def predict(text, model, tokenizer, max_length=120, top_k=7):

        encoding = tokenizer.encode_plus(
            text,
            max_length=max_length,
            return_token_type_ids=False,
            padding="max_length",
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt',
        )
        device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        model = model.to(device)
        encoding["input_ids"], encoding["attention_mask"] = encoding["input_ids"].to(device), encoding["attention_mask"].to(device)
        _, test_prediction = model(encoding["input_ids"], encoding["attention_mask"])
        top_k_values, top_k_indices = torch.topk(test_prediction, k=top_k, dim=-1)
        with open("labelencoder.json", 'r') as file:
            data = json.load(file)
        result = {}
        preds = top_k_indices.tolist()[0]
        for key, value in data.items():
            if value in preds:
                result[value] = key

        preds = list(result.values())

        return preds

In [ ]:
def predict(text, model, tokenizer, max_length=512, top_k=7):
  encoding = tokenizer.encode_plus(
    text,
    max_length=max_length,
    return_token_type_ids=False,
    padding="max_length",
    truncation=True,
    return_attention_mask=True,
    return_tensors='pt',
  )
  device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
  model = model.to(device)
  encoding["input_ids"], encoding["attention_mask"] = encoding["input_ids"].to(device), encoding["attention_mask"].to(device)
  _, test_prediction = model(encoding["input_ids"], encoding["attention_mask"])
  top_k_values, top_k_indices = torch.topk(test_prediction, k=top_k, dim=-1)
  with open("labelencoder.json", 'r') as file:
      data = json.load(file)
  result = {}
  preds = top_k_indices.tolist()[0]
  for key, value in data.items():
      if value in preds:
          result[value] = key

  preds = list(result.values())

  return preds

In [ ]:
def read_tac_hist():
    while True:
        input_string = input("Please enter proof state: ")
        input_string = input_string.upper().replace("_", "")

        if len(input_string.split()) < 3:
            print("Current minimum tactics history is 3.")
            continue  # Ask again for input
        else:
            break
    return input_string

In [ ]:
def get_recom(proof_state, roberta_model, tokenizer):
  recom = predict(proof_state, roberta_model, tokenizer)
  recomendations = []
  for string in recom:
    modified_tac = string.replace("TAC", "_TAC")
    modified_tac = modified_tac.replace("REPEAT", "REPEAT ")
    recomendations.append(modified_tac)
  print("HOL4PRS reccommendations are: ", recomendations)

# **Try HOL4PRS**



##**Loading the tokenizer and the trained model file**

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("roberta-base")
roberta_model = BertTextClassifier.load_from_checkpoint(checkpoint_path="best-checkpoint-v1.ckpt", bert_model="roberta-base", n_classes=162)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## **Demo**
In the files section on the left 📂, you've uploaded a text file containing proof samples. To test the tool, you can input a part of the proof and observe the tool's recommendation for the next step.


In [ ]:
#Enter proof state
proof_state = read_tac_hist()

Please enter proof state: ONCE_REWRITE_TAC MATCH_MP_TAC BETA_TAC


In [ ]:
#Generate recommendation
get_recom(proof_state, roberta_model, tokenizer)

HOL4PRS reccommendations are:  ['ASMSIMP_TAC', 'MESON_TAC', 'METIS_TAC', 'REPEAT STRIP_TAC', 'REWRITE_TAC', 'SET_TAC', 'SIMP_TAC']
